In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from gpt_model import OpenAIClient

openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAIClient(api_key=openai_api_key,)

In [3]:
from utils import load_data_halu_eval

halu_eval_file_path = '../data/interrogate_llm/halu_eval_long_answer.json'
output_halu_eval_file_path = '../data/output/interrogate_llm_zeroshot/halu_eval_long_answer_output.json'
reconstruct_prompt_file_path = 'prompts/reconstruct.txt'

data = load_data_halu_eval(halu_eval_file_path)

with open(reconstruct_prompt_file_path, 'r', encoding='utf-8') as f:
    reconstruct_prompt_template = f.read()

print(f"Length of HaluEval data: {len(data)}\n")
print(f"Reconstruct prompt: \n{reconstruct_prompt_template}")

Length of HaluEval data: 300

Reconstruct prompt: 
Query: You are given an answer. Your task is to generate the original question that was asked to get this answer. Make sure the original question you create explicitly includes the key information provided in the given answer. Only return the original question.

Answer: "{answer}"

Original Question:



In [4]:
from tqdm import tqdm

result = []

for idx, sample in enumerate(tqdm(data, desc="Processing data")):
    answer = sample['answer']
    reconstruct_prompt = reconstruct_prompt_template.replace('{answer}', answer)
    response = client.generate_response(reconstruct_prompt, n=5, temeprature=1)
    response_cleaned = [question.strip('"') for question in response]
    result.append(
        {
            "id": sample['id'],
            "knowledge": sample["knowledge"],
            "ground_truth": sample["ground_truth"],
            "question": sample["question"],
            "answer": sample["answer"],
            "reconstruct_questions": response_cleaned,
            "is_hallucinated": sample['is_hallucinated']
        }
    )

Processing data: 100%|██████████| 300/300 [04:16<00:00,  1.17it/s]


In [5]:
import json

with open(output_halu_eval_file_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=4)